# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>
<hr>

_Embeddings_ are representation of words thanks to vectors. These embeddings can be learnt within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which the ÌMDB dataset: it corresponds to sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2021-11-12 11:52:29.035639: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-12 11:52:29.035678: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-12 11:52:32.631555: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-12 11:52:32.631595: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-12 11:52:32.631612: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-JM9NF235): /proc/driver/nvidia/version does not exist
2021-11-12 11:52:32.631823: I tensorflow/core/platform/cpu_fe

In the previous exercise, we jointly learnt a representation for the words, and feed this representation to a RNN, as shown here : 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which can slow the convergence, and make it harder!

For that reason, we will separate the steps of learning the word representation and feeding it to a RNN. As shown here : 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with the word2vec.

The drawback is indeed that the learnt embedding is not _specifically_ designed for our task. However, learning it independently of the task at hand (sentiment analysis) has some advantages : 
- it is very fast to do in general (with word2vec)
- the representation learnt by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As on this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`.

In [7]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

Let's look at the embedded representation of some words.

You can use `word2vec.wv` as a dictionary.
For instance, `word2vec.wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation were not learn). 

In [8]:
word2vec.wv['dog']

array([-1.62191957e-01,  1.70071065e-01, -2.45468333e-01,  2.65922934e-01,
       -1.33577576e-02, -2.83126801e-01, -4.47767265e-02,  5.52905381e-01,
       -1.50194809e-01, -1.64128020e-01, -4.99876100e-04, -2.44697362e-01,
       -4.55154851e-02,  1.65142894e-01, -5.66059500e-02, -2.43938655e-01,
        1.49172276e-01, -2.39497364e-01,  4.69887629e-02, -2.25618780e-01,
        2.04116657e-01,  5.76135106e-02,  2.59595901e-01, -1.57306910e-01,
        3.26646864e-02, -1.44632757e-02, -2.53127843e-01, -4.31377143e-02,
       -1.74904794e-01,  3.47104147e-02,  1.35099351e-01,  3.26998113e-03,
        1.94032267e-01, -3.60210270e-01, -1.63180426e-01,  1.78042918e-01,
        1.57449409e-01, -1.72757998e-01, -2.53819764e-01, -3.05151284e-01,
       -2.02001259e-01, -3.45584065e-01, -2.94575095e-01,  1.67715654e-01,
        3.28958988e-01, -1.01210293e-03, -1.79165095e-01, -1.15894884e-01,
        2.93276191e-01,  2.08377019e-01,  8.45299959e-02, -1.77120194e-01,
       -2.89702207e-01, -

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [9]:
word2vec.wv['dog'].shape

(100,)

How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the `word2vec.most_similar(...)` method that, given an input word, display the "closest" words in the embedding space. If the embedding is well done, then words that have close meanings will have close representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [16]:
word2vec.wv.most_similar('home')

[('down', 0.9416477680206299),
 ('run', 0.9384677410125732),
 ('face', 0.9274929761886597),
 ('nose', 0.9257439374923706),
 ('soon', 0.922251284122467),
 ('refused', 0.9203327298164368),
 ('room', 0.9178172945976257),
 ('away', 0.9149777889251709),
 ('hospital', 0.9138537645339966),
 ('throne', 0.911676287651062)]

Similarly to `most_similar` used on words directly, we can use `similar_by_vector` on vectors to do the same thing :

In [19]:
word2vec.wv.similar_by_vector('home')

[('down', 0.9416477680206299),
 ('run', 0.9384677410125732),
 ('face', 0.9274929761886597),
 ('nose', 0.9257439374923706),
 ('soon', 0.922251284122467),
 ('refused', 0.9203327298164368),
 ('room', 0.9178172945976257),
 ('away', 0.9149777889251709),
 ('hospital', 0.9138537645339966),
 ('throne', 0.911676287651062)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [20]:
word2vec.wv['good'] - word2vec.wv['bad']

array([ 8.34465027e-06, -1.97662726e-01,  2.78807938e-01,  1.66725636e-01,
        7.67195523e-02, -3.51724088e-01, -4.91660118e-01, -1.66884810e-01,
        2.20125914e-03, -2.74118423e-01,  2.12308019e-01,  1.46225095e-01,
       -6.15184903e-02, -1.15925312e-01, -1.52898371e-01,  6.06570125e-01,
       -4.86258805e-01,  3.05060267e-01, -1.54917210e-01,  1.48749828e-01,
       -1.49787247e-01,  3.01142275e-01, -1.82030797e-01,  2.09932506e-01,
       -2.22665370e-02, -5.22472933e-02, -1.73547566e-02,  5.79126120e-01,
       -2.87116170e-01, -7.08387852e-01, -3.43976021e-01,  1.55548811e-01,
        4.67461616e-01,  1.66291326e-01, -2.27677226e-01, -2.23675370e-01,
        7.07549214e-01, -3.08950603e-01, -2.10191607e-01,  7.21844494e-01,
       -1.38853192e-01,  2.51922011e-03, -6.46763384e-01, -1.44038677e-01,
        4.74378645e-01, -2.19958022e-01,  5.46805143e-01, -9.43790674e-02,
       -1.42029226e-01,  3.11153948e-01, -2.76882797e-01, -1.54639214e-01,
       -7.15406299e-01,  

Now, image for a second that, somehow, the following equality holds true - just for a second

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to 

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be foolish of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [21]:
res = word2vec.wv['good'] - word2vec.wv['bad'] + word2vec.wv['stupid']
res

array([ 0.03391702, -0.25522515,  0.42855763, -0.18170586,  0.04509486,
       -0.8335034 , -0.7229304 ,  0.26147363, -0.460022  , -0.56117636,
       -0.01492596, -0.32755658,  0.19867742,  0.2305347 ,  0.00655818,
        0.25552237, -0.00843287, -0.2169056 , -0.28585216, -0.75041264,
        0.366194  ,  0.23525602,  0.59981865,  0.04416057, -0.17912087,
       -0.17103212, -0.31745657,  0.44983882, -0.3943409 , -0.6104598 ,
        0.39620042,  0.24805114,  0.66000116, -0.251527  , -0.4510404 ,
        0.31469542,  0.75967884, -0.18549615, -0.3969246 ,  0.01613975,
       -0.05857175, -0.5046433 , -0.5855637 ,  0.24059358,  0.705734  ,
       -0.28969383,  0.24969223, -0.40616143, -0.23292454,  0.5814529 ,
       -0.02032095, -0.18540356, -0.7342709 ,  0.42709613, -0.06254427,
        0.60365915,  0.18652192,  0.01776937, -0.41092736,  0.00667429,
        0.10861525,  0.1086095 , -0.05263376,  0.10850826, -0.4169868 ,
        0.40679464,  0.05880746, -0.14751288,  0.38543558,  0.78

We earlier said that, for any vector, it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `word2vec.wv.similar_by_vector` function) of `res`

In [22]:
word2vec.wv.similar_by_vector(res)

[('nice', 0.7686975002288818),
 ('good', 0.7466937303543091),
 ('cindy', 0.7152788043022156),
 ('such', 0.7034999132156372),
 ('spade', 0.7014268636703491),
 ('gory', 0.7013921141624451),
 ('criticized', 0.7002949714660645),
 ('tough', 0.6996453404426575),
 ('charley', 0.699077844619751),
 ('nerd', 0.6951631903648376)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [23]:
test = word2vec.wv['boy'] - word2vec.wv['girl'] + word2vec.wv['woman']

word2vec.wv.similar_by_vector(test)

[('boy', 0.9563097357749939),
 ('lady', 0.9463678002357483),
 ('child', 0.9400180578231812),
 ('career', 0.9376068711280823),
 ('sister', 0.9367451071739197),
 ('husband', 0.9356410503387451),
 ('married', 0.9355899095535278),
 ('woman', 0.934903085231781),
 ('friend', 0.9318338632583618),
 ('affair', 0.9314560890197754)]

In [24]:
word2vec.wv.similar_by_vector(word2vec.wv['queen'] - word2vec.wv['king'] + word2vec.wv['actor'])

[('actor', 0.9791503548622131),
 ('role', 0.9024245738983154),
 ('performance', 0.8922767639160156),
 ('guy', 0.8782116770744324),
 ('actress', 0.8672247529029846),
 ('job', 0.8390931487083435),
 ('character', 0.8345009088516235),
 ('pet', 0.8065794706344604),
 ('man', 0.8038941621780396),
 ('loser', 0.8018441200256348)]

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)` , actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it chooses many splits in the different sentences, choose some words as inputs $X$  and a word as output $y$ which it tries to predict, in the embedding space.

And as any neural network, Word2Vec has some hyperparameters. Let's check some. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [26]:
word2vec_2 = Word2Vec(sentences=X_train, vector_size = 50)

❓ **Question** ❓ Use the `word2vec.wv.key_to_index` attribute to display the size of the learnt vocabulary. On the other hand, compare it to the number of different words in `X_train`.

In [37]:
result = {x for l in X_train for x in l}
len(result)

30419

In [31]:
len(word2vec.wv.key_to_index)

8006

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been train on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec :  `min_count`. 

`min_count` is a integer that tells you how many occurences a given word should have to be learn in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vec that you have trained (you can choose any `vector_size` you want).

In [38]:
word2vec_3 = Word2Vec(sentences=X_train, min_count=10)

In [40]:
len(word2vec_3.wv.key_to_index)

4503

In [39]:
word2vec_4 = Word2Vec(sentences=X_train, min_count=1)

In [41]:
len(word2vec_4.wv.key_to_index)

30419

Remember that we say that word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Learn a new `word2vec_5` model with a `window` different than previously (default is 5).

In [42]:
word2vec_5 = Word2Vec(sentences=X_train, window=3)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the one that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here :

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [48]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
  embedded_sentence= []
  for w in sentence:
    if w in word2vec.wv:
        embedded_sentence.append(word2vec.wv[w])
  return np.array(embedded_sentence)

### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentence (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [49]:
def embedding(word2vec, sentences):
  embedded_sentences=[]
  for sentence in sentences:
    embedded_sentences.append(embed_sentence(word2vec,sentence))
  return np.array(embedded_sentences)

X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

/tmp/ipykernel_9289/3316067058.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(embedded_sentences)


❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad them in order to have tensors that can be divided in batch sizes during the optimization. Store the padedd values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [50]:
X_train

array([array([[ 0.4571602 , -0.46067718,  0.59801185, ..., -0.29895493,
        -0.365447  , -1.0750463 ],
       [ 0.37037343, -0.43219125,  1.418476  , ..., -1.7636448 ,
         1.309207  ,  0.74893147],
       [-0.15522805,  0.23919411, -0.52672815, ..., -0.8291645 ,
        -0.7884576 ,  0.3035907 ],
       ...,
       [-0.35189983, -0.04089173, -0.00567862, ..., -0.15471242,
         0.01800461, -0.09503834],
       [-0.10978764, -0.00903572, -0.5773599 , ..., -0.17841516,
         0.4641166 ,  0.18105611],
       [-0.00462967, -0.48991084,  0.32316735, ...,  0.08009145,
        -0.18387878, -0.8241441 ]], dtype=float32),
       array([[-0.5124969 ,  0.90683985,  2.1662226 , ...,  0.31848547,
         0.49516174, -1.1917468 ],
       [-1.1445713 , -0.57437426,  0.89105505, ..., -0.71645975,
        -0.28727973, -1.081074  ],
       [-0.48163697,  0.25392544,  1.1890442 , ..., -0.5022039 ,
        -1.0536067 , -0.15302303],
       ...,
       [ 0.6205433 , -0.06830383,  0.26571333

In [51]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post', maxlen=200)

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)